# Jupyter to .py 

Jupyter notebooks are designed for interactive computing. You type a bit of code, execute it to see the results, perhaps update the code or add new logic, execute it, test, etc. In fact, a very old name for this type of programming is "Read-Eval-Loop". The program is edited, executed (or evaluated) again and again (in a loop).

However, programs running in "production," serving APIs and clients do not need to run interactively. They need to be well tested, performant, secure pieces of code which run for days or months without interruption.

Production servers generally do not even have Jupyter installed. Programs, made up of a collection of `.py` files, along with relevant configurations and reference files make up a systemm.

Perhaps the simplest way to translate your Jupyter code is to copy and paste code cells into a .py file

**Simple ".py" file**

In [1]:
%%writefile program1.py

x = 10
print(f"The number is {x}")

Overwriting program1.py


In [40]:
%run program1.py

The number is 10


This file is run by executing the command `python program1.py`

### No automatic output

Recall that in Jupyter, the last command in a cell prints the results to screen. In a Python program, no output is printed to the console unless explicitely done so.

In [3]:
%%writefile program2.py

x = 10
y = x + x

name = "Shahbaz"
name

Overwriting program2.py


In [41]:
%run program2.py

### You must control input to the program: interactive, variables, arguments, stdin

A potentially unseen change between Jupyter and .py files is how to handle input for the program. Take the example below

In [6]:
BATCH_DATE = "1/7/2025"

In [8]:
# Some complicated program which reads sales transactions for a given day and run complicated ML models to generate complicated scores
print(f"This program is run on {BATCH_DATE}")

This program is run on 1/7/2025


If your program lives in a Jupyuter file, you can simply open the file, edit the `BATCH_DATE` field and "RUN ALL" the whole notebook.

Python programs (.py files) are absoloutely NOT edited in such a haphazard manner. Think of a program as a machine, such as a blender. In order to change the speed of a blender, you DO NOT open the blender and start shifting the gears. There is a proper and designated dial to let you control the speed. Similarly, if you want to change aspects of your code during run-time, you need to provide explicit input controls.

Extending the metaphore, **every change to a production program is akin to changing the manufacturing pipeline** for the widget. 

Here are three main ways of providing input to your program.

**Ask for input interactively**

In [10]:
%%writefile program3.py

BATCH_DATE = input("Please provide a batch date (mm/dd/yyyy)")
print(f"This program is run on {BATCH_DATE}")

Overwriting program3.py


In [43]:
%run program3.py

Please provide a batch date (mm/dd/yyyy) 1/2/3


This program is run on 1/2/3


This is almost never a good idea. This program requires a human to enter a date. What if we wanted to run this script in an automated manner? What if we wanted it to calculate the date and pass it to the program? 

Even if input is asked (such as a confirmation), you should provide an alternate to override the interactive part. A good example is Ubuntu's package manager. Here is how you install a package `apt install zip`. This command then asks for a confirmation that you, indeed, want to download and install this package. However, since this command often needs to run in an automated manner, you can also run it as `apt install -y zip`.

**Client passes in arguments**

The most common way to provide input to your program is to use _command line arguments_. Here is an example:

In [11]:
%%writefile program4.py

import sys

BATCH_DATE = sys.argv[1] #<= Here is the magic
print(f"This program is run on {BATCH_DATE}")

Writing program4.py


In [45]:
%run program4.py 1/7/2025

This program is run on 1/7/2025


Run this program as `python program4.py 1/7/2025`

Btw, what happens if you don't provide the argument? That's right, an error! **Python programs require care and _defensive_ programming**. Remember, you can't just edit the code anymore. If there is a bug or if you didn't provide enough instructions for users, the "factory" will halt and you will have to fly in to fix it (and everyone will be upset)

In [46]:
%run program4.py

IndexError: list index out of range

In [12]:
%%writefile program5.py

import sys

if len(sys.argv) < 2:
    print("Error: Missing BATCH_DATE. Please run as 'python program5.py <BATCH_DATE>`")
    sys.exit(1) # Exit with 0 indicates sucecss and 1 indicates an error

BATCH_DATE = sys.argv[1] #<= Here is the magic
print(f"This program is run on {BATCH_DATE}")

Writing program5.py


In [47]:
%run program5.py

Error: Missing BATCH_DATE. Please run as 'python program5.py <BATCH_DATE>`


SystemExit: 1

Wait, what's in `sys.argv[0]`?

This is indeed the correct way to accept input. Actually, there are several higher leve libraries which make your program much more user friendly. One, built-in library, is the argparse library

In [17]:
%%writefile program6.py

import argparse

# Create the argument parser
parser = argparse.ArgumentParser(description="Scores sales transactions to predict returns.")

# Add an argument
parser.add_argument("BATCH_DATE", type=str, help="The date of sales transactions")

# Parse the arguments
args = parser.parse_args()

# Print the argument
print(f"This program is run on {args.BATCH_DATE}")

Overwriting program6.py


In [48]:
%run program6.py

usage: program6.py [-h] BATCH_DATE
program6.py: error: the following arguments are required: BATCH_DATE


SystemExit: 2

Try running this with different inputs:
`python program6.py 1/7/2025`
`python program6.py`
`python program6.py -h`.

For very simple programs, this can be overkill.

**Environment variables**

A less common method, appropriate for certain situations, is to read in environment variables.

All computers and all running sessions have system and enviornment variables, which control a programs operate. For example, if you run the `python` command, how does your computer know where this program exists?  The answer: the `path` (or `PATH`) variable. Each operating system has its own set of variables. In mac and linux, you can view the variable by typing `echo $variable_name`. In windows you can view it by typing `echo %variable_name%`. Here is how Python can read them

In [49]:
%%writefile program7.py

import os
import sys

if len(sys.argv) < 2:
    print("Error: Missing enviornment variable name. Please run as 'python program5.py <ENV>`")
    sys.exit(1)
    
ENV = sys.argv[1]
ENV_VAL = os.environ.get(ENV, "")

print(f"Environment variable {ENV} has value {ENV_VAL}")

Overwriting program7.py


Try this with arguments `path`. You can find other variables with the `set` command (different operating systems may use different commands).

Users can set their own enviornment variables. They can be permanent and globally avialable or temporary and only available in a single termiinal session. For example, set a custom variable and try running the program again:

Windows: `SET SERVER_TYPE=PRODUCTION`
Linux/mac: `SERVER_TYPE=PRODUCTION`

Now try running the earlier program and look for the value SERVER_TYPE. Start a new terminal and try running the same program with the same argument.

An important usecase of environment variables is to identify the context in which the program is running. Machines can be set to `PROD`, `STAGING`, `DEV` and the program can decide how critical sections are run.

Please note that although this can be useful, this is not a common way of reading input.